# Updated notebook for covid19

#### Covid19 Coronairus Cases Data and Visualization
* Data obtained from Johns Hopkins (refreshed periodically)
* US Data at both county level and state level (so need to dedup to calculate total)
* Worldwide data mostly at country level, except US, China, Canada, Australia at state/province level
* Original data: all cumulative: confirmed, death, recovered. 
* Also Chinese Data from Ding Xiang Yuan which provide city level data for China.

* Real Time Dashboard here: https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6


## Data Structures
* gbl_conf_aggr : global cum confirmed aggregated by country. Index == country. First column is 1/24 data
* gbl_conf_incr : global daily confirmed aggregated by country. Index == country. First column is 1/24 data
* us_conf_aggr :  US State confirmed aggregated by state. Index == state. First column is 1/24 data
* us_conf_incr :  US State daily confirmed aggregated by state. Index == state. First column is 1/24 data
* county_conf_aggr :  County confirmed aggregated by CombinedKey Index == county. First column is 1/24 data
* county_conf_incr :  County daily confirmed aggregated by CombinedKey. Index == county. First column is 1/24 data



In [2]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sbn
import plotly  as py
import math
import datetime
import sys
import os

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='iframe'  # notebook, jupyterlab, ...
pd.set_option('display.max_rows', None)


In [3]:
import us_state
from us_state import *
import covid_util
from   covid_util import datastr_to_datetime, convert_date

In [4]:
GBL_CONF_CSV = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
US_CONF_CSV =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"

In [5]:
# Province/State	Country/Region	Lat	Long	1/22/20	1/23/20	
gbl_conf_df = pd.read_csv(GBL_CONF_CSV)

# State	Country	1/22/20	1/23/20
gbl_conf_df.rename(columns = { 'Province/State' : 'State',  'Country/Region' : "Country"},  inplace = True)
gbl_conf_df.drop(  ['Lat', 'Long'], axis=1, inplace=True) 
gbl_conf_df.head()

,State,Country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582
1,NaN,Albania,0,0,0,0,0,0,0,0,...,916,933,946,948,949,964,969,981,989,998
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,6629,6821,7019,7201,7377,7542,7728,7918,8113,8306
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,761,761,761,761,761,762,762,762,762,762
4,NaN,Angola,0,0,0,0,0,0,0,0,...,48,48,48,50,52,52,58,60,61,69


In [6]:
# UID	iso2	iso3	code3	FIPS	Admin2	Province_State	Country_Region	Lat	Long_	...	4/10/20)
us_conf_df  = pd.read_csv(US_CONF_CSV)

# County	State	Country	1/22/20	1/23/20
us_conf_df.rename(columns = { 'Province_State' : 'State', 'Admin2': 'County', 'Country_Region' : "Country"}, inplace = True)
#us_conf_df.drop( ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Lat', 'Long_', 'Combined_Key'], axis = 1, inplace=True)
us_conf_df.drop( ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Lat', 'Long_', 'Country' ], axis = 1, inplace=True)
us_conf_df.head(10)

,County,State,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20
0,NaN,American Samoa,"American Samoa, US",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Guam,"Guam, US",0,0,0,0,0,0,0,...,154,154,154,154,154,154,165,165,165,166
2,NaN,Northern Mariana Islands,"Northern Mariana Islands, US",0,0,0,0,0,0,0,...,19,21,21,21,21,21,22,22,22,22
3,NaN,Puerto Rico,"Puerto Rico, US",0,0,0,0,0,0,0,...,2542,2589,2646,2710,2805,2866,2913,3030,3100,3189
4,NaN,Virgin Islands,"Virgin Islands, US",0,0,0,0,0,0,0,...,69,69,69,69,69,69,69,69,69,69
5,Autauga,Alabama,"Autauga, Alabama, US",0,0,0,0,0,0,0,...,103,110,110,120,127,136,143,149,155,159
6,Baldwin,Alabama,"Baldwin, Alabama, US",0,0,0,0,0,0,0,...,244,254,254,260,262,270,269,271,273,274
7,Barbour,Alabama,"Barbour, Alabama, US",0,0,0,0,0,0,0,...,79,79,81,85,90,96,99,104,105,110
8,Bibb,Alabama,"Bibb, Alabama, US",0,0,0,0,0,0,0,...,49,50,50,50,51,52,52,55,58,59
9,Blount,Alabama,"Blount, Alabama, US",0,0,0,0,0,0,0,...,45,45,46,47,47,47,47,49,49,49


In [7]:
print("Shape of gbl_conf_df", gbl_conf_df.shape)
print("Shape of us_conf_df",  us_conf_df.shape)
us_conf_df.head()
#gbl_conf_df.head()

Shape of gbl_conf_df (266, 126)
Shape of us_conf_df (3261, 127)


,County,State,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20
0,NaN,American Samoa,"American Samoa, US",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Guam,"Guam, US",0,0,0,0,0,0,0,...,154,154,154,154,154,154,165,165,165,166
2,NaN,Northern Mariana Islands,"Northern Mariana Islands, US",0,0,0,0,0,0,0,...,19,21,21,21,21,21,22,22,22,22
3,NaN,Puerto Rico,"Puerto Rico, US",0,0,0,0,0,0,0,...,2542,2589,2646,2710,2805,2866,2913,3030,3100,3189
4,NaN,Virgin Islands,"Virgin Islands, US",0,0,0,0,0,0,0,...,69,69,69,69,69,69,69,69,69,69


## Aggregate by Country for global, and state for US

In [8]:
gbl_conf_aggr =  gbl_conf_df.groupby('Country').sum()  #as_inxde
gbl_conf_aggr[15:20]

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20
Country,,,,,,,,,,,,,,,,,,,,,
Belarus,0,0,0,0,0,0,0,0,0,0,...,27730,28681,29650,30572,31508,32426,33371,34303,35244,36198
Belgium,0,0,0,0,0,0,0,0,0,0,...,54644,54989,55280,55559,55791,55983,56235,56511,56810,57092
Belize,0,0,0,0,0,0,0,0,0,0,...,18,18,18,18,18,18,18,18,18,18
Benin,0,0,0,0,0,0,0,0,0,0,...,339,339,339,339,130,130,135,135,135,191
Bhutan,0,0,0,0,0,0,0,0,0,0,...,21,21,21,21,21,21,21,21,24,24


In [9]:
us_conf_aggr =  us_conf_df.groupby('State').sum()  #as_inxde
us_conf_aggr[15:20]
county_conf_aggr =  us_conf_df.groupby('Combined_Key').sum()
county_conf_aggr[ county_conf_aggr.index == 'Santa Clara, California, US']

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20
Combined_Key,,,,,,,,,,,,,,,,,,,,,
"Santa Clara, California, US",0,0,0,0,0,0,0,0,0,1,...,2391,2418,2453,2453,2483,2492,2492,2492,2571,2617


## Derive the Daily Increment

In [10]:
COL_LIST = gbl_conf_aggr.columns
def get_delta_df(df, offset = 0):
    new_df = df.copy(deep=True)
    prev_col = None
    for col in COL_LIST:
        #print(type(col), str(col))
        if prev_col is None:
          prev_col = col
          continue
        new_df[col] = df[col] - df[prev_col]
        prev_col = col
    return(new_df)

gbl_conf_incr = get_delta_df(gbl_conf_aggr, offset = 0)
us_conf_incr =  get_delta_df(us_conf_aggr, offset = 0)
county_conf_incr = get_delta_df(county_conf_aggr, offset = 0) 

print(gbl_conf_incr.shape, gbl_conf_aggr.shape)
print(us_conf_incr.shape, us_conf_aggr.shape)
print(county_conf_incr.shape, county_conf_aggr.shape)
print(gbl_conf_aggr[35:39][-4:])
print(gbl_conf_incr[35:39][-4:])
print(county_conf_incr[ county_conf_incr.index == 'Santa Clara, California, US'][-10:]) 


(188, 124) (188, 124)
(58, 124) (58, 124)
(3261, 124) (3261, 124)
          1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country                                                                   
Chile           0        0        0        0        0        0        0   
China         548      643      920     1406     2075     2877     5509   
Colombia        0        0        0        0        0        0        0   
Comoros         0        0        0        0        0        0        0   

          1/29/20  1/30/20  1/31/20  ...  5/15/20  5/16/20  5/17/20  5/18/20  \
Country                              ...                                       
Chile           0        0        0  ...    39542    41428    43781    46059   
China        6087     8141     9802  ...    84038    84044    84054    84063   
Colombia        0        0        0  ...    14216    14939    15574    16295   
Comoros         0        0        0  ...       11       11       11       11   

  

## Data Conversion

In [11]:
COL_LIST = gbl_conf_aggr.columns
DT_LIST = convert_date(COL_LIST)
dt_list = DT_LIST
# DT_LIST

In [12]:
#DT_LIST

## Plotting 

In [13]:
## Helper routine to do the acutal plotting
def plot_bar(X, Y, name='', title = ''): 
    fig = go.Figure()
    fig.add_trace(go.Bar( x = X, y = Y, name=name)) 
    if title != '':
        fig.update_layout(title = title)
    fig.show()
    return(fig)

def plot_line(X, Z, name='', color = 'Green'):
    fig = go.Figure()
    fig.add_trace(go.Scatter( x = X, y = Z, name=name, line_color=color))
    fig.show()
    return(fig)

def plot_bar_line(X, Y, Z,  name_y='Daily Incr', name_z='Cumulative', title='', color_z='Green', logy=False, show_cum=True):
    fig = go.Figure()
    pio.renderers.default='iframe'
    fig.add_trace(go.Bar( x = X, y = Y,  name=name_y ))
    if show_cum:
      fig.add_trace(go.Scatter( x = X, y = Z,  name=name_z, line_color=color_z))
    fig.update_layout(title = title)
    if logy:
        fig.update_layout( yaxis_type="log")
    fig.show()
    return(fig)

def plot_entity_util(Y, Z, name_y='Daily Incr', name_z='Cumulative', title='', 
                          color_z='Green', show_cum = True, logy = False, lastn = 0, fname=None ):
    X = DT_LIST
    if lastn != 0:
        X = X[lastn: ]  # date
        Y = Y[lastn: ]  # typically daily incr
        Z = Z[lastn: ]  # typicall cumulative
    fig = plot_bar_line(X, Y, Z, name_y=name_y, name_z=name_z, 
                  title=title, color_z=color_z, show_cum=show_cum, logy=logy  ) 
    if fname is not None:
        save_image(fig, fname)
    return(fig)



In [14]:
## Plot country or state confirmed cases

def plot_entity_confirmed(y, z, entity, show_cum = True, logy = False, lastn = 0, title=None, fname = None, retItem=False):
  try:
    if y is None or z is None:
      print("Invalid country name provided: %s" % entity)
    if title is None:
        title = '%s Confirmed Cases (as of %s)' % (entity, COL_LIST[-1])
    fig = plot_entity_util(y, z, show_cum=show_cum, logy=logy, lastn=lastn, fname=fname, title=title)
    if retItem:
      return(y, z)
    else:
      return
  except Exception as e:
    print("Exception: invalid entity name provided: %s" % entity)
    print(str(e))
    return(None, None)

def plot_country_confirmed(Country, show_cum = True, logy = False, lastn = 0, fname = None, title=None, retItem=False):
    y = gbl_conf_incr[ gbl_conf_incr.index == Country].iloc[0]
    z = gbl_conf_aggr[ gbl_conf_aggr.index == Country].iloc[0]
    if y is None or z is None:
        print("Input entity name %s cannot be found" % Country)
        return
    return plot_entity_confirmed(y, z, entity=Country, 
                                 lastn = lastn, show_cum = show_cum, logy = logy, title=title, fname = fname, retItem=False)

def plot_state_confirmed(State, show_cum = True, logy = False, lastn = 0, fname = None, title=None, retItem=False):
    y = us_conf_incr[ us_conf_incr.index == State].iloc[0]
    z = us_conf_aggr[ us_conf_aggr.index == State].iloc[0]
    if y is None or z is None:
        print("Input entity name %s cannot be found" % State)
        return
    return plot_entity_confirmed(y, z, entity=State, 
                                 lastn = lastn, show_cum = show_cum, logy = logy, title=title, fname = fname, retItem=False)

def plot_county_confirmed(County, show_cum = True, logy = False, lastn = 0, fname = None, title=None, retItem=False):
    y = county_conf_incr[ county_conf_incr.index == County].iloc[0]
    z = county_conf_aggr[ county_conf_aggr.index == County].iloc[0]
    if y is None or z is None:
        print("Input entity name %s cannot be found" % County)
        return
    return plot_entity_confirmed(y, z, entity=County, 
                                 lastn = lastn, show_cum = show_cum, logy = logy, title=title, fname = fname, retItem=False)
 
#plot_country_confirmed('Italy')
# gbl_conf_aggr[ gbl_conf_aggr.index == 'Italy'].iloc[0]
# gbl_conf_incr[ gbl_conf_incr.index == 'Italy'].iloc[0]
                          
#OK plot_country_confirmed('US',  lastn = -60,  show_cum = True, logy=True)
plot_state_confirmed('New York',  lastn = -60,  show_cum = True, logy=True)
#plot_state_confirmed('New York',  lastn = -45,  show_cum = False, logy=False)
#plot_state_confirmed('New Jersey',  lastn = -45,  show_cum = False, logy=False)
#plot_state_confirmed('Connecticut',  lastn = -45,  show_cum = False, logy=False)
#plot_state_confirmed('Washington',  lastn = -120,  show_cum = True, logy=False)


In [15]:
from plotly.subplots import make_subplots
def plot_4entity(y1, z1, txt1, y2, z2, txt2, y3, z3, txt3, y4, z4, txt4, show_cum=True, logy=False, lastn=0):
  dt_range = dt_list[lastn: ]
  X = dt_range
  fig = make_subplots(rows=2, cols=2, subplot_titles = (txt1, txt2, txt3, txt4)) 
  fig.add_trace(go.Bar(x=X, y=y1, name='incr' ), row=1, col=1)
  fig.add_trace(go.Bar(x=X, y=y2, name='incr' ), row=1, col=2)
  fig.add_trace(go.Bar(x=X, y=y3, name='incr' ), row=2, col=1)
  fig.add_trace(go.Bar(x=X, y=y4, name='incr' ), row=2, col=2)
  fig.add_trace(go.Scatter(x=X, y=z1, name='cum' ), row=1, col=1)
  fig.add_trace(go.Scatter(x=X, y=z2, name='cum' ), row=1, col=2)
  fig.add_trace(go.Scatter(x=X, y=z3, name='cum' ), row=2, col=1)
  fig.add_trace(go.Scatter(x=X, y=z4, name='cum' ), row=2, col=2)
  if logy:
        fig.update_layout( yaxis_type="log")
  fig.update_layout( title_text="COVID-19 Daily Increment and Cumulative Confirmed Cases as of %s " % str(COL_LIST[-1:].values) )
  fig.show()

In [16]:
def plot_4_entity_confirmed(incr_df, aggr_df,  Clist, show_cum=True, logy=False, lastn=0):
  C1 = Clist[0]; C2 = Clist[1]; C3 = Clist[2];  C4 = Clist[3];    
  try:
    y1 = incr_df[ incr_df.index == C1].iloc[0]
    y2 = incr_df[ incr_df.index == C2].iloc[0]
    y3 = incr_df[ incr_df.index == C3].iloc[0]
    y4 = incr_df[ incr_df.index == C4].iloc[0]
    if lastn != 0:
        y1 = y1[lastn: ]
        y2 = y2[lastn: ]
        y3 = y3[lastn: ]
        y4 = y4[lastn: ]
    if show_cum:
      z1 = aggr_df[ aggr_df.index == C1].iloc[0]
      z2 = aggr_df[ aggr_df.index == C2].iloc[0]
      z3 = aggr_df[ aggr_df.index == C3].iloc[0]
      z4 = aggr_df[ aggr_df.index == C4].iloc[0]
      if lastn != 0:
            z1 = z1[lastn: ]
            z2 = z2[lastn: ]
            z3 = z3[lastn: ]
            z4 = z4[lastn: ]
    else:
       (z1, z2, z3, z4) = (None, None, None, None)
    
        
    plot_4entity( y1, z1, "%s" % C1, 
                  y2, z2, "%s" % C2,
                  y3, z3, "%s" % C3,
                  y4, z4, "%s" % C4,
                  logy = logy, show_cum = show_cum, lastn = lastn)
  except Exception as e:
    print("Exception: ", str(e))
    
Countries = ['Spain', 'Germany', 'Italy', 'France']    
plot_4_entity_confirmed(gbl_conf_incr, gbl_conf_aggr,  Countries,  logy=True, show_cum=False, lastn=-45 )

#Countries = ['Netherlands', 'Belgium', 'Sweden', 'Denmark']    
Countries = ['Netherlands', 'Germany', 'Sweden', 'Italy']    
plot_4_entity_confirmed(gbl_conf_incr,gbl_conf_aggr,  Countries,  logy=False, show_cum=False, lastn=-45 )

States    = ['New York', 'New Jersey', 'Connecticut', 'California']    
#plot_4_entity_confirmed(gbl_conf_incr,us_conf_aggr,  States, logy=True, show_cum=True, lastn=-45 )


In [17]:
States    = ['New York', 'New Jersey', 'Connecticut', 'California']    
plot_4_entity_confirmed(us_conf_incr, us_conf_aggr,  States, logy=False, show_cum=False, lastn=-45 )

In [18]:
# NYC vs non-NYC area
# ch2 = ch1[ ch1.State.apply(lambda x : x in['Hong Kong','Macau', 'Hubei', 'Guangdong', 'Beijing', 'Shanghai'] ) ]    
print(us_conf_aggr.shape)
nyc3_conf_aggr_df = us_conf_aggr.loc[ ['New York', 'New Jersey', 'Connecticut'], : ]
nyc3_conf_aggr_df.shape
nyc3_conf_aggr = nyc3_conf_aggr_df.sum()
nyc3_conf_aggr

nyc3_conf_incr_df = us_conf_incr.loc[ ['New York', 'New Jersey', 'Connecticut'], : ]
nyc3_conf_incr_df.shape
nyc3_conf_incr = nyc3_conf_incr_df.sum()
#nyc3_conf_incr


(58, 124)


In [19]:

us_confirmed =  gbl_conf_aggr.loc[ 'US', :]
us_daily =  gbl_conf_incr.loc[ 'US', :]
non_nyc3_conf_aggr = np.subtract(us_confirmed, nyc3_conf_aggr)
non_nyc3_conf_incr = np.subtract(us_daily, nyc3_conf_incr)

#print( non_nyc3_conf_aggr[-10:], nyc3_conf_aggr[-10:], us_confirmed[-10:])
plot_entity_confirmed(non_nyc3_conf_incr, non_nyc3_conf_aggr, 'US except NYC', show_cum = False, logy = False, lastn = -45, title=None, fname = None, retItem=False)
plot_entity_confirmed(nyc3_conf_incr, nyc3_conf_aggr, 'NYC', show_cum = False, logy = False, lastn = -45, title=None, fname = None, retItem=False)


## Calculate Numberic Table

In [20]:
def calc_multiple(incr_df, aggr_df, entity_list, entity_type='country', last_weeks=None, last_days = None):
    if last_days is not None:
      lastn = last_days
    else:
      lastn = 7 * last_weeks -1
    midpoint = int((-1 + lastn)/2 )
    df = pd.DataFrame(columns = ['Entity', 'Num_Begin', 'Num_Mid', 'Num_End', 'Delta1', 'Delta2',
                  
                                 'Ratio_m2b', 'Ratio_e2m', 'Ratio_e2b', 'WCGrowth', 'Ratio_w2w'])
    
    col_list = aggr_df.columns
    print("%20s,   %s,  %s,    %s,   ratio1,  ratio2, rate of growth per week "  % 
          ('Entity', COL_LIST[lastn], COL_LIST[midpoint], COL_LIST[-1]))
    for entity in entity_list:
      try:  
        ts = aggr_df[aggr_df.index == entity].iloc[0]
      except Exception as e:
        print("Exception in processing entity = ", entity)
        print(str(e))
        continue
      num_start = ts[ col_list[lastn:lastn+1]].iloc[0]
      num_mid =   ts[ col_list[midpoint:midpoint+1]].iloc[0] 
      num_end   = ts[col_list[-1:]].iloc[0]
      ratio1 = float(num_mid) / num_start
      ratio2 = float(num_end) / num_mid
      ratio3 = float(num_end) / num_start
      delta1 = int(num_mid - num_start)
      delta2 = int(num_end - num_mid)
      if delta1 == 0:
         delta1 = 1
      ratio4 = float(delta2)/ float(delta1)
        
      root  = math.sqrt(float(num_end/num_start))  
      #print("%20s, begin=%5d,  mid=%5d,  end=%5d,  ratio1=%4.1f, ratio2=%4.1f, ratio3=%4.1f, root=%4.1fX" %
      #      (country, num_start, num_mid, num_end, ratio1, ratio2, ratio3, root) )
      df = df.append( {'Entity':entity, 
                       'Num_Begin':num_start, 'Num_Mid': num_mid, 'Num_End':num_end, 
                       'Ratio_m2b':"%3.2f" % ratio1,  'Ratio_e2m':"%3.2f" % ratio2, 
                       'Ratio_e2b':"%3.2f" % ratio3,
                       'WCGrowth': "%3.2f" % root,
                       'Delta1':"%d" % delta1,  'Delta2':"%d" % delta2, 
                       'Ratio_w2w':"%3.2f"%ratio4 }, ignore_index=True) 
    return(df)
'UK', 
CTop_list3 = ['US', 'Spain', 'Italy', 'France', 'Germany', 'Turkey', 'United Kingdom', 'Canada', 'Australia', 'China']
df3 = calc_multiple(gbl_conf_incr, gbl_conf_aggr,  CTop_list3, last_days = -15 )
df3

              Entity,   5/10/20,  5/17/20,    5/24/20,   ratio1,  ratio2, rate of growth per week 


,Entity,Num_Begin,Num_Mid,Num_End,Delta1,Delta2,Ratio_m2b,Ratio_e2m,Ratio_e2b,WCGrowth,Ratio_w2w
0,US,1329260,1486757,1643246,157497,156489,1.12,1.11,1.24,1.11,0.99
1,Spain,224350,230698,235772,6348,5074,1.03,1.02,1.05,1.03,0.80
2,Italy,219070,225435,229858,6365,4423,1.03,1.02,1.05,1.02,0.69
3,France,177094,179693,182709,2599,3016,1.01,1.02,1.03,1.02,1.16
4,Germany,171879,176369,180328,4490,3959,1.03,1.02,1.05,1.02,0.88
5,Turkey,138657,149435,156827,10778,7392,1.08,1.05,1.13,1.06,0.69
6,United Kingdom,220449,244995,260916,24546,15921,1.11,1.06,1.18,1.09,0.65
7,Canada,70091,78332,86106,8241,7774,1.12,1.10,1.23,1.11,0.94
8,Australia,6948,7054,7114,106,60,1.02,1.01,1.02,1.01,0.57
9,China,84010,84054,84095,44,41,1.00,1.00,1.00,1.00,0.93


## Calc State and County

In [22]:
STop_list3 = [ 'California', 'Washington', 'New York', 'Connecticut', 'New Jersey', 'Pennsylvania', 
              'Georgia', 'Florida', 'Texas',
              'Maryland', 'Virginia',  'District of Columbia' ] 
calc_multiple(us_conf_incr, us_conf_aggr, STop_list3, last_days = -15 )

              Entity,   5/10/20,  5/17/20,    5/24/20,   ratio1,  ratio2, rate of growth per week 


,Entity,Num_Begin,Num_Mid,Num_End,Delta1,Delta2,Ratio_m2b,Ratio_e2m,Ratio_e2b,WCGrowth,Ratio_w2w
0,California,67600,80166,94020,12566,13854,1.19,1.17,1.39,1.18,1.10
1,Washington,16891,18433,19828,1542,1395,1.09,1.08,1.17,1.08,0.90
2,New York,335395,350121,361515,14726,11394,1.04,1.03,1.08,1.04,0.77
3,Connecticut,33554,37419,40468,3865,3049,1.12,1.08,1.21,1.10,0.79
4,New Jersey,138754,146504,154154,7750,7650,1.06,1.05,1.11,1.05,0.99
5,Pennsylvania,59939,65700,71563,5761,5863,1.10,1.09,1.19,1.09,1.02
6,Georgia,33580,37579,42902,3999,5323,1.12,1.14,1.28,1.13,1.33
7,Florida,40596,45588,50867,4992,5279,1.12,1.12,1.25,1.12,1.06
8,Texas,39258,48396,55861,9138,7465,1.23,1.15,1.42,1.19,0.82
9,Maryland,32587,38804,46313,6217,7509,1.19,1.19,1.42,1.19,1.21


In [32]:
# EXtended Bay Area County: including Sacramento, Stockton, Santa Cruz 
SFBay_list3 = ['Santa Clara, California, US', 
          'San Mateo, California, US', 
          'San Francisco, California, US', 
          'Alameda, California, US', 
          'Contra Costa, California, US', 
          'Marin, California, US', 
          'Sonoma, California, US', 
          'Napa, California, US', 
          'Solano, California, US', 
          'Santa Cruz, California, US', 
          'Yolo, California, US', 
          'Sacramento, California, US', 
          'Kern, California, US', 
          'San Joaquin, California, US'
         ]
calc_multiple(county_conf_incr, county_conf_aggr, SFBay_list3, last_days = -15 )
#county_conf_aggr

              Entity,   4/19/20,  4/26/20,    5/3/20,   ratio1,  ratio2, rate of growth per week 


,Entity,Num_Begin,Num_Mid,Num_End,Delta1,Delta2,Ratio_m2b,Ratio_e2m,Ratio_e2b,WCGrowth,Ratio_w2w
0,"Santa Clara, California, US",1870,2084,2231,214,147,1.11,1.07,1.19,1.09,0.69
1,"San Mateo, California, US",838,1019,1233,181,214,1.22,1.21,1.47,1.21,1.18
2,"San Francisco, California, US",1157,1408,1602,251,194,1.22,1.14,1.38,1.18,0.77
3,"Alameda, California, US",1164,1470,1749,306,279,1.26,1.19,1.50,1.23,0.91
4,"Contra Costa, California, US",693,817,945,124,128,1.18,1.16,1.36,1.17,1.03
5,"Marin, California, US",189,223,240,34,17,1.18,1.08,1.27,1.13,0.50
6,"Sonoma, California, US",180,218,252,38,34,1.21,1.16,1.40,1.18,0.89
7,"Napa, California, US",47,60,70,13,10,1.28,1.17,1.49,1.22,0.77
8,"Solano, California, US",169,199,266,30,67,1.18,1.34,1.57,1.25,2.23
9,"Santa Cruz, California, US",104,120,137,16,17,1.15,1.14,1.32,1.15,1.06


# Dashoard

In [33]:
#plot_entity_confirmed(non_nyc3_conf_incr, non_nyc3_conf_aggr, 'US except NYC', show_cum = False, logy = False, lastn = -45, title=None, fname = None, retItem=False)
#plot_entity_confirmed(nyc3_conf_incr, nyc3_conf_aggr, 'NYC', show_cum = False, logy = False, lastn = -45, title=None, fname = None, retItem=False)
plot_county_confirmed('Santa Clara, California, US', show_cum = True, logy = False, lastn = -45, title=None, fname = None, retItem=False)
plot_country_confirmed('US', show_cum = True, logy = True, lastn = -45, title=None, fname = None, retItem=False)


In [34]:
CountryA = 'Turkey'
plot_country_confirmed(CountryA, show_cum = True, logy = True, lastn = -45, title=None, fname = None, retItem=False)


In [35]:
States    = ['New York', 'New Jersey', 'Connecticut', 'California']    
plot_4_entity_confirmed(us_conf_incr, us_conf_aggr,  States, logy=False, show_cum=False, lastn=-45 )

In [36]:
cdf3 = calc_multiple(gbl_conf_incr, gbl_conf_aggr,  CTop_list3, last_days = -15 )

cdf3

              Entity,   4/19/20,  4/26/20,    5/3/20,   ratio1,  ratio2, rate of growth per week 


,Entity,Num_Begin,Num_Mid,Num_End,Delta1,Delta2,Ratio_m2b,Ratio_e2m,Ratio_e2b,WCGrowth,Ratio_w2w
0,US,758809,965785,1158040,206976,192255,1.27,1.20,1.53,1.24,0.93
1,Spain,198674,207634,217466,8960,9832,1.05,1.05,1.09,1.05,1.10
2,Italy,178972,197675,210717,18703,13042,1.10,1.07,1.18,1.09,0.70
3,France,153011,162220,168925,9209,6705,1.06,1.04,1.10,1.05,0.73
4,Germany,145184,157770,165664,12586,7894,1.09,1.05,1.14,1.07,0.63
5,Turkey,86306,110130,126045,23824,15915,1.28,1.14,1.46,1.21,0.67
6,United Kingdom,121172,154037,187842,32865,33805,1.27,1.22,1.55,1.25,1.03
7,Canada,35633,48033,60504,12400,12471,1.35,1.26,1.70,1.30,1.01
8,Australia,6610,6714,6822,104,108,1.02,1.02,1.03,1.02,1.04
9,China,83805,83912,83964,107,52,1.00,1.00,1.00,1.00,0.49


In [39]:
sdf3 = calc_multiple(us_conf_incr, us_conf_aggr,  STop_list3, last_days = -15 )
sdf3

              Entity,   4/19/20,  4/26/20,    5/3/20,   ratio1,  ratio2, rate of growth per week 
Exception in processing entity =  Pennyslvania
single positional indexer is out-of-bounds


,Entity,Num_Begin,Num_Mid,Num_End,Delta1,Delta2,Ratio_m2b,Ratio_e2m,Ratio_e2b,WCGrowth,Ratio_w2w
0,California,31431,43558,54903,12127,11345,1.39,1.26,1.75,1.32,0.94
1,Washington,11978,13521,15185,1543,1664,1.13,1.12,1.27,1.13,1.08
2,New York,243382,288045,316415,44663,28370,1.18,1.10,1.30,1.14,0.64
3,Connecticut,17962,25269,29287,7307,4018,1.41,1.16,1.63,1.28,0.55
4,New Jersey,85301,109038,126744,23737,17706,1.28,1.16,1.49,1.22,0.75
5,Georgia,18301,23486,28665,5185,5179,1.28,1.22,1.57,1.25,1.00
6,Florida,26314,31532,36078,5218,4546,1.20,1.14,1.37,1.17,0.87
7,Texas,19260,24967,31998,5707,7031,1.30,1.28,1.66,1.29,1.23
8,Maryland,12847,18581,25462,5734,6881,1.45,1.37,1.98,1.41,1.20
9,Virginia,8542,12970,18672,4428,5702,1.52,1.44,2.19,1.48,1.29


In [40]:

pdf3 = calc_multiple(county_conf_incr, county_conf_aggr, SFBay_list3, last_days = -15 )
pdf3

              Entity,   4/19/20,  4/26/20,    5/3/20,   ratio1,  ratio2, rate of growth per week 


,Entity,Num_Begin,Num_Mid,Num_End,Delta1,Delta2,Ratio_m2b,Ratio_e2m,Ratio_e2b,WCGrowth,Ratio_w2w
0,"Santa Clara, California, US",1870,2084,2231,214,147,1.11,1.07,1.19,1.09,0.69
1,"San Mateo, California, US",838,1019,1233,181,214,1.22,1.21,1.47,1.21,1.18
2,"San Francisco, California, US",1157,1408,1602,251,194,1.22,1.14,1.38,1.18,0.77
3,"Alameda, California, US",1164,1470,1749,306,279,1.26,1.19,1.50,1.23,0.91
4,"Contra Costa, California, US",693,817,945,124,128,1.18,1.16,1.36,1.17,1.03
5,"Marin, California, US",189,223,240,34,17,1.18,1.08,1.27,1.13,0.50
6,"Sonoma, California, US",180,218,252,38,34,1.21,1.16,1.40,1.18,0.89
7,"Napa, California, US",47,60,70,13,10,1.28,1.17,1.49,1.22,0.77
8,"Solano, California, US",169,199,266,30,67,1.18,1.34,1.57,1.25,2.23
9,"Santa Cruz, California, US",104,120,137,16,17,1.15,1.14,1.32,1.15,1.06
